In [1]:
import numpy as np
import pandas as pd
import re
import requests
import json
import time


In [2]:
df = pd.read_csv("4. Final_Dataset.csv")
df.head(2)

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
0,6,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,Battery life is only 6 hours,35,battery life is only 6 hours,"Based on the given examples and rules, here's ...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,['Battery - Short backup time'],['Battery - Short backup time']
1,12,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Starlight|Size: 16GB Unified Memory|St...,False,I am planning to but this model but I am seein...,10,i am planning to but this model but i am seein...,"Based on the review content, here's the output...","{'Sentiment': 'Neutral', 'Aspect-PainPoint Pai...",Neutral,['Processor - Low speed (0.01mhz)'],['Processor - Slow performance']


### Data Preprocessing

In [3]:
df.columns

Index(['Index', 'ASIN', 'Model Number', 'Manufacturer Name', 'Generic Name',
       'Product Title', 'Price', 'Overall Rating', 'Rating Count',
       'Review Count', 'Bought L Month', 'Product Link', 'Customer Say',
       'Insight', 'Individual Rating', 'Review Title', 'Review Date',
       'Product Description', 'Verified Purchase', 'Review Text',
       'Helpful Votes', 'Cleaned Review Text', 'llm_raw', 'llm_parsed',
       'Sentiment', 'Aspect-PainPoint Pairs',
       'Standardized Aspect-PainPoint Pairs'],
      dtype='object')

## 🔄 Exploding Aspect-PainPoint Pairs into Individual Rows

This block transforms the list-style `"Aspect-PainPoint Pairs"` and `"Standardized Aspect-PainPoint Pairs"` into **row-wise entries** — allowing easier downstream filtering, grouping, or labeling.

In [4]:
final_list = []
last_len = 0

def change_quotes(x):
    return x.replace('"', "'") if isinstance(x, str) else x

df['Aspect-PainPoint Pairs'] = df['Aspect-PainPoint Pairs'].apply(change_quotes)
df['Standardized Aspect-PainPoint Pairs'] = df['Standardized Aspect-PainPoint Pairs'].apply(change_quotes)

for idx, rows in df.iterrows():
    a1 = rows["Aspect-PainPoint Pairs"].strip('[').strip(']').split("',")
    a2 = rows["Standardized Aspect-PainPoint Pairs"].strip('[').strip(']').split("',")

    last_len = max(len(a1), len(a2))

    for first in range(last_len):
        x1 = a1[first].strip().strip("'") if first < len(a1) else None
        x2 = a2[first].strip().strip("'") if first < len(a2) else None

        final_list.append([
            rows['Index'], rows['ASIN'], rows['Model Number'], rows['Manufacturer Name'], rows['Generic Name'], 
            rows['Product Title'], rows['Price'], rows['Overall Rating'], rows['Rating Count'], rows['Review Count'], 
            rows['Bought L Month'], rows['Product Link'], rows['Customer Say'], rows['Insight'], rows['Individual Rating'], 
            rows['Review Title'], rows['Review Date'], rows['Product Description'], rows['Verified Purchase'], 
            rows['Review Text'], rows['Helpful Votes'], rows['Cleaned Review Text'], rows['llm_raw'], rows['llm_parsed'], 
            rows['Sentiment'], x1, x2
        ])

In [5]:
df1 = pd.DataFrame(data = final_list, columns=['Index', 'ASIN', 'Model Number', 'Manufacturer Name', 'Generic Name',
       'Product Title', 'Price', 'Overall Rating', 'Rating Count',
       'Review Count', 'Bought L Month', 'Product Link', 'Customer Say',
       'Insight', 'Individual Rating', 'Review Title', 'Review Date',
       'Product Description', 'Verified Purchase', 'Review Text',
       'Helpful Votes', 'Cleaned Review Text', 'llm_raw', 'llm_parsed',
       'Sentiment', 'Aspect-PainPoint Pairs',
       'Standardized Aspect-PainPoint Pairs'])



In [6]:
df1.head()

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
0,6,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,Battery life is only 6 hours,35,battery life is only 6 hours,"Based on the given examples and rules, here's ...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,Battery - Short backup time,Battery - Short backup time
1,12,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Starlight|Size: 16GB Unified Memory|St...,False,I am planning to but this model but I am seein...,10,i am planning to but this model but i am seein...,"Based on the review content, here's the output...","{'Sentiment': 'Neutral', 'Aspect-PainPoint Pai...",Neutral,Processor - Low speed (0.01mhz),Processor - Slow performance
2,17,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...",47390,3.9,609,165,...,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,So 1st of all I got this laptop in 41000 durin...,99,so 1st of all i got this laptop in 41000 durin...,What a long review!\n\nHere's my analysis:\n\n...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,Display - Not OLED/WUXGA quality,Display - Poor display quality
3,17,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...",47390,3.9,609,165,...,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,So 1st of all I got this laptop in 41000 durin...,99,so 1st of all i got this laptop in 41000 durin...,What a long review!\n\nHere's my analysis:\n\n...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,Battery - Short backup time (2-6 hours dependi...,Battery - Limited battery life
4,17,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...",47390,3.9,609,165,...,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,So 1st of all I got this laptop in 41000 durin...,99,so 1st of all i got this laptop in 41000 durin...,What a long review!\n\nHere's my analysis:\n\n...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,Processor - Low gaming performance,Processor - Gaming performance issues


In [7]:
df1.shape

(3798, 27)

In [8]:
df1.isnull().sum()

Index                                     0
ASIN                                      0
Model Number                              0
Manufacturer Name                         0
Generic Name                             38
Product Title                             0
Price                                     0
Overall Rating                            0
Rating Count                              0
Review Count                              0
Bought L Month                         1260
Product Link                              0
Customer Say                            476
Insight                                 476
Individual Rating                         1
Review Title                              0
Review Date                               0
Product Description                     644
Verified Purchase                         0
Review Text                               0
Helpful Votes                             0
Cleaned Review Text                       0
llm_raw                         

In [9]:
df1[
    (df1['Aspect-PainPoint Pairs'].isnull() | df1['Aspect-PainPoint Pairs'].isin(["None", ""]))
    &
    (df1['Standardized Aspect-PainPoint Pairs'].isnull() | df1['Standardized Aspect-PainPoint Pairs'].isin(["None", ""]))
]

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
5,18,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...",47390,3.9,609,165,...,Colour: i5-12th | Office 2024,True,"This Laptop is great, No issues. but I want to...",1,"this laptop is great, no issues. but i want to...",A new challenge!\n\nAfter analyzing the review...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,None
233,289,B08N5W4NNB,MGN63HN/A,Apple,Laptop,"Apple MacBook Air Laptop: Apple M1 chip, 13.3-...",54990,4.6,7004,1500,...,Colour: Space Grey|Size: 256 GB,True,This is the review i am writing after using th...,5,this is the review i am writing after using th...,"### Analyzing the review...\n\n{\n ""Sentiment...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,None
243,317,B08N5W4NNB,MGN63HN/A,Apple,Laptop,"Apple MacBook Air Laptop: Apple M1 chip, 13.3-...",54990,4.6,7004,1500,...,Colour: Silver|Size: 256 GB,True,I got my product delivered earlier this week. ...,95,i got my product delivered earlier this week. ...,A new review to analyze!\n\nAfter processing t...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,None
347,426,B0F4R6H3NB,fd0577TU,HP,Laptop,"HP 15, 13th Gen Intel Core i5-1335U (16GB DDR4...",51990,3.8,26,18,...,Size: 512GB,False,Product design is good. Performance is stable....,0,product design is good. performance is stable....,"The review says: """"""product design is good. pe...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,None
514,592,B0C4ZM63RP,Inspiron 3530,Dell,Laptop,"Dell Inspiron 3530 Thin & Light Laptop, 13th G...",33990,4.0,171,48,...,Colour: Black,True,worst and no proper communication for refund t...,6,worst and no proper communication for refund t...,"Based on the provided examples, I will extract...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,None,None
1264,1432,B0DRCTY5JH,15IGL7,Lenovo,Laptop,Lenovo IdeaPad Slim 1 Intel® Celeron® N4020 15...,25500,3.6,320,74,...,Size: 4GB | 256GB SSD,True,The lappy is performing extremely slow even on...,3,the lappy is performing extremely slow even on...,Here's the output for the given review:\n\n{\n...,"{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,None,None
1430,1568,B0F8WC83WT,HP 15,HP,BUSINESS LAPTOP,"HP 15 (2025), Intel Core i3 13th Gen 1315U - (...",33490,4.0,19,12,...,NaN,False,"good enough value for simple tasks, it doesn't...",0,"good enough value for simple tasks, it doesnt ...","A new review to analyze!\n\nReview: """"""good en...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,None
1666,1860,B0DJCWZ5BT,245 G9,HP,BUSINESS LAPTOP,HP Laptop 245 G9 AMD Ryzen 3 3250U Dual Core -...,26490,3.1,23,13,...,NaN,True,Product comes without in built MS office. It a...,1,product comes without in built ms office. it a...,"The review is:\n\n""product comes without in bu...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,None,None
1746,1943,B0C3RF3HT3,15-fd0006TU,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U (8GB DDR4,...",35690,3.8,235,90,...,Size: 8GB Ram,True,It got damaged with in one year . Worst quality,0,it got damaged with in one year . worst quality,Based on the provided information and examples...,"{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,None,None
1868,2086,B0CY2LDT1S,14IMH9,Lenovo,Laptop,"Lenovo IdeaPad Pro 5, Intel Core Ultra 9 185H,...",108940,4.2,141,65,...,"Colour: 14"" | 2.8K OLED",True,Received 5 months used laptop. Ab aap dekh lo ...,4,received 5 months used laptop. ab aap dekh lo ...,"I can help with that!\n\nReview: """"""received 5...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,None,None


In [10]:
x = df1[
    (df1['Aspect-PainPoint Pairs'].isnull() | df1['Aspect-PainPoint Pairs'].isin(["None", ""]))
    &
    (df1['Standardized Aspect-PainPoint Pairs'].isnull() | df1['Standardized Aspect-PainPoint Pairs'].isin(["None", ""]))
].index
x

Index([   5,  233,  243,  347,  514, 1264, 1430, 1666, 1746, 1868, 2276, 2515,
       2583, 2600, 2952, 2964, 3489, 3542, 3637],
      dtype='int64')

In [11]:
df1.shape

(3798, 27)

In [12]:
df1.drop(x, axis=0, inplace=True)

In [13]:
df1.shape

(3779, 27)

In [14]:
df1[
    (df1['Aspect-PainPoint Pairs'].isnull() | df1['Aspect-PainPoint Pairs'].isin(["None", ""]))
    &
    (df1['Standardized Aspect-PainPoint Pairs'].isnull() | df1['Standardized Aspect-PainPoint Pairs'].isin(["None", ""]))
]

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs


In [15]:
df1.isnull().sum()

Index                                     0
ASIN                                      0
Model Number                              0
Manufacturer Name                         0
Generic Name                             38
Product Title                             0
Price                                     0
Overall Rating                            0
Rating Count                              0
Review Count                              0
Bought L Month                         1251
Product Link                              0
Customer Say                            475
Insight                                 475
Individual Rating                         1
Review Title                              0
Review Date                               0
Product Description                     641
Verified Purchase                         0
Review Text                               0
Helpful Votes                             0
Cleaned Review Text                       0
llm_raw                         

In [16]:
df1[(df1['Aspect-PainPoint Pairs'].isnull()) | (df1['Aspect-PainPoint Pairs'].isin(["None", "", "none"]))]

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
986,1143,B0D872NJSM,15IAU7,Lenovo,Laptop,Lenovo IdeaPad 3 12th Gen Intel Core i3-1215U ...,34539,4.1,1145,208,...,Style Name: Intel Core i3-11th Gen,True,The laptop is excellent with respect to perfor...,5,the laptop is excellent with respect to perfor...,"Based on the review content, here's the output...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,None,Display - May not be suitable for those specif...


> **Note:** `index != False` is intentionally set to retain the DataFrame index as a column.  
> This index acts as a **unique primary key** to help trace each review across different transformations and outputs.


In [18]:
# df1.to_csv('5. Analyze_dataset1.csv')

In [19]:
df1 = pd.read_csv('5. Analyze_dataset1.csv')

In [20]:
df1.columns

Index(['Primary Key', 'Index', 'ASIN', 'Model Number', 'Manufacturer Name',
       'Generic Name', 'Product Title', 'Price', 'Overall Rating',
       'Rating Count', 'Review Count', 'Bought L Month', 'Product Link',
       'Customer Say', 'Insight', 'Individual Rating', 'Review Title',
       'Review Date', 'Product Description', 'Verified Purchase',
       'Review Text', 'Helpful Votes', 'Cleaned Review Text', 'llm_raw',
       'llm_parsed', 'Sentiment', 'Aspect-PainPoint Pairs',
       'Standardized Aspect-PainPoint Pairs'],
      dtype='object')

In [21]:
crop_df = df1[['Primary Key', 'Aspect-PainPoint Pairs', 'Standardized Aspect-PainPoint Pairs']]
crop_df.head(2)

,Primary Key,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
0,0,Battery - Short backup time,Battery - Short backup time
1,1,Processor - Low speed (0.01mhz),Processor - Slow performance


In [22]:
# crop_df.to_csv('5_1. review_df.csv', index=False)

> 📌 To ensure the accuracy of the standardized dataset, I exported it as **`review_df.csv`** and manually verified the transformations using **Power Query Editor**.  
> After confirming the data consistency and correctness, I saved the final version as **`Main Dataset.xlsx`** for further analysis and reporting.


In [25]:
data = pd.read_excel("5_2. Main Dataset.xlsx")
data.head(2)

,Primary Key,Original Aspect-PainPoint,Standardized Category,Standardized Aspect,Standardized Pain Point
0,0,Battery - Short backup time,Battery - Short Backup Time,Battery,Short Backup Time
1,1,Processor - Low speed (0.01mhz),Processor - Slow Performance,Processor,Slow Performance


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3778 entries, 0 to 3777
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Primary Key                3778 non-null   int64 
 1   Original Aspect-PainPoint  3778 non-null   object
 2   Standardized Category      3778 non-null   object
 3   Standardized Aspect        3778 non-null   object
 4   Standardized Pain Point    3775 non-null   object
dtypes: int64(1), object(4)
memory usage: 147.7+ KB


In [27]:
len(data)

3778

In [28]:
df1.columns

Index(['Primary Key', 'Index', 'ASIN', 'Model Number', 'Manufacturer Name',
       'Generic Name', 'Product Title', 'Price', 'Overall Rating',
       'Rating Count', 'Review Count', 'Bought L Month', 'Product Link',
       'Customer Say', 'Insight', 'Individual Rating', 'Review Title',
       'Review Date', 'Product Description', 'Verified Purchase',
       'Review Text', 'Helpful Votes', 'Cleaned Review Text', 'llm_raw',
       'llm_parsed', 'Sentiment', 'Aspect-PainPoint Pairs',
       'Standardized Aspect-PainPoint Pairs'],
      dtype='object')

In [29]:
data.columns

Index(['Primary Key', 'Original Aspect-PainPoint', 'Standardized Category',
       'Standardized Aspect', 'Standardized Pain Point'],
      dtype='object')

In [31]:
data.duplicated(subset=['Primary Key']).sum()

0

In [32]:
data[data.duplicated(subset=['Primary Key'], keep = False)].sort_values(by=['Primary Key']).head(5)

,Primary Key,Original Aspect-PainPoint,Standardized Category,Standardized Aspect,Standardized Pain Point


In [33]:
len(data)

3778

In [34]:
len(df1)

3779

In [35]:
ddf = pd.merge(df1, data, on=['Primary Key'], how='inner')
ddf.head(2)

,Primary Key,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,...,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs,Original Aspect-PainPoint,Standardized Category,Standardized Aspect,Standardized Pain Point
0,0,6,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,...,battery life is only 6 hours,"Based on the given examples and rules, here's ...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,Battery - Short backup time,Battery - Short backup time,Battery - Short backup time,Battery - Short Backup Time,Battery,Short Backup Time
1,1,12,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,...,i am planning to but this model but i am seein...,"Based on the review content, here's the output...","{'Sentiment': 'Neutral', 'Aspect-PainPoint Pai...",Neutral,Processor - Low speed (0.01mhz),Processor - Slow performance,Processor - Low speed (0.01mhz),Processor - Slow Performance,Processor,Slow Performance


In [36]:
len(ddf)

3778

In [37]:
# ddf.to_csv("6. my_dataset.csv", index=False)